# Chapter 8 - Unsupervised Learning - K-Means
The following consists of a minimal implementation of the K-Means algorithm, as well as a plotting function to show the different steps performed by the algorithms from initialization to convergence.

In [1]:
import sys
sys.path.append("../")
from utils import *

In [2]:
def animate_kmeans(X, est, save_to = None, frame_duration=1000):
    import plotly.express as px
    colors = np.array(["lightgrey"] + px.colors.qualitative.Plotly)
    
    frames = []
    frames.append(go.Frame(data=[go.Scatter(x=X[:,0], y=X[:,1], mode="markers", marker=dict(symbol="circle-open",color=colors[est.assignments[0]+1]), name="Data-points"),
                                 go.Scatter(x=est.centroids[0][:,0], y=est.centroids[0][:,1], mode="markers", marker=dict(symbol="x",color=colors[1:], size=10), name="Centroids")],
                       layout=go.Layout(title_text="Initialization")))
    
    for i in range(1, len(est.centroids)-1):          
        frames.append(go.Frame(data=[go.Scatter(x=X[:,0], y=X[:,1], mode="markers", marker=dict(symbol="circle-open",color=colors[est.assignments[i]+1]),name="Data-points"),
                         go.Scatter(x=est.centroids[i-1][:,0], y=est.centroids[i-1][:,1], mode="markers", marker=dict(symbol="x",color=colors[1:], size=10), name="Centroids")],
                   layout=go.Layout(title_text=f"Iteration {i} - Assignment Step")))
        
        title = f"Converged - Required {len(est.centroids)-2 } Iterations - Final Cost {round(est.costs[i], 3)}" if i == len(est.centroids)-2 else \
                f"Iteration {i} - Update Centroids Step - Cost {round(est.costs[i], 3)}"
        
        frames.append(go.Frame(data=[go.Scatter(x=X[:,0], y=X[:,1], mode="markers", marker=dict(symbol="circle-open",color=colors[est.assignments[i]+1]), name="Data-points"),
                                     go.Scatter(x=est.centroids[i][:,0], y=est.centroids[i][:,1], mode="markers", marker=dict(symbol="x",color=colors[1:], size=10), name="Centroids", )],
                               layout=go.Layout(title_text=title)))
        
    
    fig = go.Figure(data=frames[0]["data"],
            frames=frames,
            layout=go.Layout(
                title=frames[0]["layout"]["title"],
                updatemenus=[dict(visible=True,
                                  type="buttons",
                                  buttons=[dict(label="Play",
                                                method="animate",
                                                args=[None, dict(frame={"duration":frame_duration}) ]),
                                          
                                          dict(label="Pause",
                                               method="animate",
                                               args=[[None], {"frame": {"duration": 0, "redraw": False},
                                                              "mode": "immediate",
                                                              "transition": {"duration": 0}}])])]  ))
    
    if save_to:
        animation_to_gif(fig, save_to, frame_duration) 
    return fig

### Basic Scenario
In the following we generate a dataset from 4 Gaussians with equal covarianec matrix. In addition, the number of samples drawn from each Gaussian is identical.

In [3]:
X = np.concatenate([np.random.multivariate_normal([0, 0], [[1,0],[0,1]], 30),
                    np.random.multivariate_normal([6,0], [[1,0],[0,1]], 30),
                    np.random.multivariate_normal([2,4], [[1,0],[0,1]], 30),
                    np.random.multivariate_normal([5.5,5], [[1,0],[0,1]], 30)])
animate_kmeans(X, K_Means(4).fit(X), frame_duration=1000, save_to="../kmeans-basic.gif")

NameError: name 'K_Means' is not defined

### Multiple Optimal Solutions Scenario
Using a similar dataset to the one above, we will show how for a given centroids choice the algorithm could converge to more than a single solution. Notice that if the `min`/`argmin` operations are stable then as long as the data-points are given in the same order, only one of the optimal solutions will be returned.

In [5]:
X = np.array([[-1,0,0,1],[0,-1,1,0]]).T
X = np.concatenate([X + [0, 0], X + [10, 10], X + [10, 0], X + [0, 10]])

centroids = np.array([[5,5],[5,5]])

animate_kmeans(X, K_Means(initial_centroids=centroids, tie_breaking="random").fit(X), 
               frame_duration=500, save_to="../kmeans-multiple-a.gif")
animate_kmeans(X, K_Means(initial_centroids=centroids, tie_breaking="random").fit(X), 
               frame_duration=500, save_to="../kmeans-multiple-b.gif")

### Sub-optimal Solution Scenario

In [6]:
X = np.array([[-1,0,0,1],[0,-1,1,0]]).T
X = np.concatenate([X + [0, 0], X + [20, 10], X + [20, 0], X + [0, 10]])

centroids = np.array([[10,6],[10,4]])

animate_kmeans(X, K_Means(initial_centroids=centroids, tie_breaking="random").fit(X), 
               frame_duration=500, save_to="../kmeans-suboptimal.gif")

### Hyper-parameter Selection
Using `sklearn`'s K-Means implementation, we demonstrate selecting the number of clusters using the elbow-plot approach

In [7]:
from sklearn.cluster import KMeans

X = np.concatenate([np.random.multivariate_normal([0, 0], [[1,0],[0,1]], 30),
                    np.random.multivariate_normal([6,0], [[1,0],[0,1]], 30),
                    np.random.multivariate_normal([2,4], [[1,0],[0,1]], 30),
                    np.random.multivariate_normal([5.5,5], [[1,0],[0,1]], 30)])

ks = list(range(1, 21))
costs = [KMeans(n_clusters=k, init="random").fit(X).inertia_ for k in ks]

fig = make_subplots(1, 2)
fig.add_trace(go.Scatter(x=X[:,0], y=X[:,1], mode="markers", marker=dict(color="black", opacity=.7), showlegend=False), row=1, col=1)
fig.add_trace(go.Scatter(x=ks, y=costs, mode="markers+lines", showlegend=False), row=1, col=2)
fig.update_layout(height=300, width=800, title_text=r"$\text{Sum Of Squared Distances As Function Of }k$")
fig.write_image("../kmeans-selecting-k.png")
fig.show()


C:\Users\gilad\anaconda3\envs\iml.env\lib\site-packages\sklearn\cluster\_kmeans.py:882: UserWarning:

KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.

